## Full code

In [ ]:
import os
import cv2
import glob
import utils
import numpy as np
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted

from deepface.detectors import RetinaFaceWrapper

In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

In [ ]:
models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib",
]
model_name2 = 'enet_b2_8_best'

In [ ]:
basepath = os.getcwd()
list_vid = glob.glob(basepath+'/visual-features/train/*')
list_vid = natsorted(list_vid)

In [ ]:
# Iterate through videos in new_list_vid with a progress bar
for vid in tqdm(new_list_vid, desc='Extracting Features'):
    # Get the base name of the video file
    base_vid = Path(vid).stem
    # Create a destination directory for the extracted features
    desdir = os.path.join(basepath, 'visual-features', 'facial-train', base_vid)
    if not os.path.isdir(desdir):
        os.makedirs(desdir)
    
    # Get all jpg frames from the video directory
    frame_paths = glob.glob(vid + '/*.jpg')
    # Sort the frame paths in natural order
    frame_paths = natsorted(frame_paths)
    
    # Process each frame in the video
    for f, fp in enumerate(frame_paths):
        # Get the base name of the frame
        base_frame = Path(fp).stem
        # Read the image and convert from BGR to RGB
        im = cv2.imread(fp)[:, :, ::-1]
        # Ensure the image is contiguous in memory
        im = np.ascontiguousarray(im, dtype=np.uint8)
        
        # Detect faces in the image
        faces = RetinaFaceWrapper.detect_face(img=im, face_detector=None)
        femb1 = []
        
        # Extract features for each detected face
        for i in range(len(faces)):
            embedding1 = utils.represent_from_face(face_img=faces[i][0][0],
                  model_name = models[2],
                  normalization='Facenet',
            )
            femb1.append(embedding1)
        
        # Convert the list of embeddings to a numpy array
        array_1 = np.array(femb1, dtype=np.float32)
        # Save the compressed numpy array of face embeddings
        np.savez_compressed(desdir+f'/{base_frame}', fr=array_1)